In [ ]:
!pip install ray

     |████████████████████████████████| 22.0MB 1.8MB/s 
     |████████████████████████████████| 1.2MB 38.1MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 204kB 42.3MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 2.9MB 41.5MB/s 
     |████████████████████████████████| 133kB 48.0MB/s 
     |████████████████████████████████| 266kB 48.6MB/s 
     |████████████████████████████████| 153kB 49.4MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12622 sha256=c20eecafc2ba34c2e572ac6208457be3e67f95748808e46279ca467c180f33f6
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=

In [ ]:
%matplotlib inline


Parameter Server
================

The parameter server is a framework for distributed machine learning training.

In the parameter server framework, a centralized server (or group of server
nodes) maintains global shared parameters of a machine-learning model
(e.g., a neural network) while the data and computation of calculating
updates (i.e., gradient descent updates) are distributed over worker nodes.

![](../images/param_actor.png)

    :align: center

Parameter servers are a core part of many machine learning applications. This
document walks through how to implement simple synchronous and asynchronous
parameter servers using Ray actors.

To run the application, first install some dependencies.

.. code-block:: bash

  pip install torch torchvision filelock

Let's first define some helper functions and import some dependencies.


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from filelock import FileLock
import numpy as np

import ray


def get_data_loader():
    """Safely downloads data. Returns training/validation set dataloader."""
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    # We add FileLock here because multiple workers will want to
    # download data, and this may cause overwrites since
    # DataLoader is not threadsafe.
    with FileLock(os.path.expanduser("~/data.lock")):
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST(
                "~/data",
                train=True,
                download=True,
                transform=mnist_transforms),
            batch_size=128,
            shuffle=True)
        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST("~/data", train=False, transform=mnist_transforms),
            batch_size=128,
            shuffle=True)
    return train_loader, test_loader


def evaluate(model, test_loader):
    """Evaluates the accuracy of the model on a validation dataset."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            # This is only set to finish evaluation faster.
            if batch_idx * len(data) > 1024:
                break
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100. * correct / total

Setup: Defining the Neural Network
----------------------------------

We define a small neural network to use in training. We provide
some helper functions for obtaining data, including getter/setter
methods for gradients and weights.



In [ ]:
class ConvNet(nn.Module):
    """Small ConvNet for MNIST."""

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

    def get_weights(self):
        return {k: v.cpu() for k, v in self.state_dict().items()}

    def set_weights(self, weights):
        self.load_state_dict(weights)

    def get_gradients(self):
        grads = []
        for p in self.parameters():
            grad = None if p.grad is None else p.grad.data.cpu().numpy()
            grads.append(grad)
        return grads

    def set_gradients(self, gradients):
        for g, p in zip(gradients, self.parameters()):
            if g is not None:
                p.grad = torch.from_numpy(g)

Defining the Parameter Server
-----------------------------

The parameter server will hold a copy of the model.
During training, it will:

1. Receive gradients and apply them to its model.

2. Send the updated model back to the workers.

The ``@ray.remote`` decorator defines a remote process. It wraps the
ParameterServer class and allows users to instantiate it as a
remote actor.



In [ ]:
@ray.remote
class ParameterServer(object):
    def __init__(self, lr):
        self.model = ConvNet()
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=lr)

    def apply_gradients(self, *gradients):
        summed_gradients = [
            np.stack(gradient_zip).sum(axis=0)
            for gradient_zip in zip(*gradients)
        ]
        self.optimizer.zero_grad()
        self.model.set_gradients(summed_gradients)
        self.optimizer.step()
        return self.model.get_weights()

    def get_weights(self):
        return self.model.get_weights()

Defining the Worker
-------------------
The worker will also hold a copy of the model. During training. it will
continuously evaluate data and send gradients
to the parameter server. The worker will synchronize its model with the
Parameter Server model weights.



In [ ]:
@ray.remote
class DataWorker(object):
    def __init__(self):
        self.model = ConvNet()
        self.data_iterator = iter(get_data_loader()[0])

    def compute_gradients(self, weights):
        self.model.set_weights(weights)
        try:
            data, target = next(self.data_iterator)
        except StopIteration:  # When the epoch ends, start a new epoch.
            self.data_iterator = iter(get_data_loader()[0])
            data, target = next(self.data_iterator)
        self.model.zero_grad()
        output = self.model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        return self.model.get_gradients()

Synchronous Parameter Server Training
-------------------------------------
We'll now create a synchronous parameter server training scheme. We'll first
instantiate a process for the parameter server, along with multiple
workers.



In [ ]:
iterations = 200
num_workers = 2

ray.init(ignore_reinit_error=True)
ps = ParameterServer.remote(1e-2)
workers = [DataWorker.remote() for i in range(num_workers)]

2020-09-09 15:54:01,681	INFO resource_spec.py:231 -- Starting Ray with 7.23 GiB memory available for workers and up to 3.62 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-09 15:54:02,127	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


We'll also instantiate a model on the driver process to evaluate the test
accuracy during training.



In [ ]:
model = ConvNet()
test_loader = get_data_loader()[1]

Training alternates between:

1. Computing the gradients given the current weights from the server
2. Updating the parameter server's weights with the gradients.



In [ ]:
print("Running synchronous parameter server training.")
current_weights = ps.get_weights.remote()
for i in range(iterations):
    gradients = [
        worker.compute_gradients.remote(current_weights) for worker in workers
    ]
    # Calculate update after all gradients are available.
    current_weights = ps.apply_gradients.remote(*gradients)

    if i % 10 == 0:
        # Evaluate the current model.
        model.set_weights(ray.get(current_weights))
        accuracy = evaluate(model, test_loader)
        print("Iter {}: \taccuracy is {:.1f}".format(i, accuracy))

print("Final accuracy is {:.1f}.".format(accuracy))
# Clean up Ray resources and processes before the next example.
ray.shutdown()

Running synchronous parameter server training.


2020-09-09 15:54:06,126	WARNING worker.py:1134 -- The dashboard on node f93c97114d15 failed with the following error:
Traceback (most recent call last):
  File "/usr/lib/python3.6/asyncio/base_events.py", line 1062, in create_server
    sock.bind(sa)
OSError: [Errno 99] Cannot assign requested address

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ray/dashboard/dashboard.py", line 961, in <module>
    dashboard.run()
  File "/usr/local/lib/python3.6/dist-packages/ray/dashboard/dashboard.py", line 576, in run
    aiohttp.web.run_app(self.app, host=self.host, port=self.port)
  File "/usr/local/lib/python3.6/dist-packages/aiohttp/web.py", line 433, in run_app
    reuse_port=reuse_port))
  File "/usr/lib/python3.6/asyncio/base_events.py", line 484, in run_until_complete
    return future.result()
  File "/usr/local/lib/python3.6/dist-packages/aiohttp/web.py", line 359, in _run_app
    a

Iter 0: 	accuracy is 12.5
Iter 10: 	accuracy is 24.7
Iter 20: 	accuracy is 38.5
Iter 30: 	accuracy is 54.5
Iter 40: 	accuracy is 67.4
Iter 50: 	accuracy is 73.5
Iter 60: 	accuracy is 77.3
Iter 70: 	accuracy is 79.3
Iter 80: 	accuracy is 80.1
Iter 90: 	accuracy is 84.3
Iter 100: 	accuracy is 85.2
Iter 110: 	accuracy is 84.5
Iter 120: 	accuracy is 84.9
Iter 130: 	accuracy is 84.9
Iter 140: 	accuracy is 87.1
Iter 150: 	accuracy is 85.9
Iter 160: 	accuracy is 85.4
Iter 170: 	accuracy is 87.8
Iter 180: 	accuracy is 88.6
Iter 190: 	accuracy is 87.8
Final accuracy is 87.8.


Asynchronous Parameter Server Training
--------------------------------------
We'll now create a synchronous parameter server training scheme. We'll first
instantiate a process for the parameter server, along with multiple
workers.



In [ ]:
print("Running Asynchronous Parameter Server Training.")

ray.init(ignore_reinit_error=True)
ps = ParameterServer.remote(1e-2)
workers = [DataWorker.remote() for i in range(num_workers)]

2020-09-09 15:54:28,938	INFO resource_spec.py:231 -- Starting Ray with 7.18 GiB memory available for workers and up to 3.61 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


Running Asynchronous Parameter Server Training.


2020-09-09 15:54:29,374	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


Here, workers will asynchronously compute the gradients given its
current weights and send these gradients to the parameter server as
soon as they are ready. When the Parameter server finishes applying the
new gradient, the server will send back a copy of the current weights to the
worker. The worker will then update the weights and repeat.



In [ ]:
current_weights = ps.get_weights.remote()

gradients = {}
for worker in workers:
    gradients[worker.compute_gradients.remote(current_weights)] = worker

for i in range(iterations * num_workers):
    ready_gradient_list, _ = ray.wait(list(gradients))
    ready_gradient_id = ready_gradient_list[0]
    worker = gradients.pop(ready_gradient_id)

    # Compute and apply gradients.
    current_weights = ps.apply_gradients.remote(*[ready_gradient_id])
    gradients[worker.compute_gradients.remote(current_weights)] = worker

    if i % 10 == 0:
        # Evaluate the current model after every 10 updates.
        model.set_weights(ray.get(current_weights))
        accuracy = evaluate(model, test_loader)
        print("Iter {}: \taccuracy is {:.1f}".format(i, accuracy))

print("Final accuracy is {:.1f}.".format(accuracy))

Iter 0: 	accuracy is 7.8
Iter 10: 	accuracy is 11.7
Iter 20: 	accuracy is 21.9
Iter 30: 	accuracy is 32.4
Iter 40: 	accuracy is 46.9
Iter 50: 	accuracy is 54.6
Iter 60: 	accuracy is 63.9
Iter 70: 	accuracy is 67.6
Iter 80: 	accuracy is 70.7
Iter 90: 	accuracy is 71.8
Iter 100: 	accuracy is 75.3
Iter 110: 	accuracy is 78.5
Iter 120: 	accuracy is 81.1
Iter 130: 	accuracy is 81.2
Iter 140: 	accuracy is 81.0
Iter 150: 	accuracy is 82.9
Iter 160: 	accuracy is 81.7
Iter 170: 	accuracy is 80.0
Iter 180: 	accuracy is 83.6
Iter 190: 	accuracy is 83.8
Iter 200: 	accuracy is 83.8
Iter 210: 	accuracy is 84.7
Iter 220: 	accuracy is 85.0
Iter 230: 	accuracy is 86.5
Iter 240: 	accuracy is 85.3
Iter 250: 	accuracy is 85.5
Iter 260: 	accuracy is 85.8
Iter 270: 	accuracy is 85.1
Iter 280: 	accuracy is 86.5
Iter 290: 	accuracy is 87.2
Iter 300: 	accuracy is 86.5
Iter 310: 	accuracy is 86.5
Iter 320: 	accuracy is 87.1
Iter 330: 	accuracy is 88.0
Iter 340: 	accuracy is 86.6
Iter 350: 	accuracy is 88.7
Iter

Final Thoughts
--------------

This approach is powerful because it enables you to implement a parameter
server with a few lines of code as part of a Python application.
As a result, this simplifies the deployment of applications that use
parameter servers and to modify the behavior of the parameter server.

For example, sharding the parameter server, changing the update rule,
switch between asynchronous and synchronous updates, ignoring
straggler workers, or any number of other customizations,
will only require a few extra lines of code.



In [ ]:
import time
@ray.remote

def fu(n):
  time.sleep(n)
  return n

resl = []
for i in range(3):
  resl.append(ray.get(fu.remote(i)))
print(resl)

[0, 1, 2]


In [ ]:
def funPrint(x):
  print(len(x))
  for i in range(len(x)):
    print(x[i])




max_iteration = 90
sample_size = 3000
Rc = 20
r = 0.4
theta = [0.3, 0.4, 0.3]
states = np.zeros([max_iteration+1,sample_size],dtype=int)
action = np.zeros([max_iteration,sample_size],dtype=int)

for ss in range(sample_size):
  aId = 0
  for mi in range(max_iteration):
    ds = np.random.choice([0,1,2],p=theta)
    states[mi+1][ss] = states[mi][ss] +ds
    if aId==0:
      a0 = states[mi+1][ss] * r
      if a0 < Rc:
        action[mi][ss] = 0
      else:
        action[mi][ss] = 1
        aId = mi +1
    else:
      a0 = (states[mi+1][ss] - states[aId][ss])* r
      if a0 < Rc:
        action[mi][ss] = 0
      else:
        action[mi][ss] = 1
        aId = mi +1
funPrint(states)
funPrint(action)      

91
[0 0 0 ... 0 0 0]
[0 0 0 ... 1 1 1]
[2 1 2 ... 3 1 3]
[3 1 3 ... 5 1 4]
[5 3 4 ... 5 2 4]
[6 5 5 ... 7 3 5]
[7 7 6 ... 9 4 7]
[ 8  9  6 ... 10  6  8]
[ 8 10  8 ... 11  7  9]
[ 9 10  9 ... 13  7 10]
[ 9 12  9 ... 13  7 12]
[10 13  9 ... 14  7 12]
[12 13  9 ... 16  9 13]
[12 15 11 ... 17 10 13]
[14 15 12 ... 19 10 15]
[16 15 13 ... 19 11 17]
[18 16 15 ... 20 13 19]
[20 16 16 ... 21 15 21]
[20 18 16 ... 23 17 21]
[20 18 18 ... 24 17 22]
[21 20 20 ... 26 18 23]
[23 22 21 ... 27 18 23]
[24 23 21 ... 28 20 23]
[25 24 22 ... 28 21 24]
[26 24 23 ... 30 21 26]
[27 24 24 ... 31 21 26]
[28 24 26 ... 32 21 27]
[28 25 27 ... 33 22 27]
[29 27 29 ... 33 23 29]
[30 28 31 ... 34 24 30]
[30 30 32 ... 35 24 30]
[31 32 32 ... 37 26 32]
[33 34 32 ... 38 28 34]
[35 36 34 ... 40 29 34]
[36 37 35 ... 41 30 34]
[37 38 35 ... 43 31 34]
[37 40 35 ... 44 32 34]
[37 40 37 ... 46 32 35]
[39 42 37 ... 46 32 35]
[40 42 38 ... 48 34 36]
[42 42 39 ... 48 35 37]
[42 42 40 ... 48 35 38]
[44 44 40 ... 49 35 40]
[44 46 

In [ ]:
!pip install ray

     |████████████████████████████████| 22.0MB 61.9MB/s 
     |████████████████████████████████| 2.9MB 44.1MB/s 
     |████████████████████████████████| 133kB 37.3MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 204kB 42.1MB/s 
     |████████████████████████████████| 1.2MB 42.4MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 266kB 35.5MB/s 
     |████████████████████████████████| 153kB 44.5MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12622 sha256=ff7a7727b2201985f5894eafc055fb76f74416b31179b2e77b9e482f87ee53c3
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size

In [ ]:
import ray

@ray.remote
def tryFunc(*x):
  print(x[0])
  print(x[0])

  print(x[0])

  print(x[0])

  print(x[0])

  return x[1]
ray.shutdown()
ray.init(num_cpus=3, ignore_reinit_error=True, log_to_driver=False)


haha = [tryFunc.remote(*[i,i]) for i in range(3)]
print(ray.get(haha))
print(haha)
#results = ray.get,([tryFunc.remote(*[i]) for i in range(3)])

2020-09-16 15:56:21,900	INFO resource_spec.py:231 -- Starting Ray with 7.28 GiB memory available for workers and up to 3.65 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-16 15:56:22,292	INFO services.py:1193 -- View the Ray dashboard at localhost:8265
2020-09-16 15:56:25,884	WARNING worker.py:1134 -- The dashboard on node 46743d90a7b7 failed with the following error:
Traceback (most recent call last):
  File "/usr/lib/python3.6/asyncio/base_events.py", line 1062, in create_server
    sock.bind(sa)
OSError: [Errno 99] Cannot assign requested address

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ray/dashboard/dashboard.py", line 961, in <module>
    dashboard.run()
  File "/usr/local/lib/python3.6/dist-packages/ray/dashboard/dashboard.py", line 576, in run
    aiohttp.web.run_app(self.app, host=self.host, port=self.

[0, 1, 2]
[ObjectRef(8e3e479219c9e512ffffffff010000c001000000), ObjectRef(169cae522c26b747ffffffff010000c001000000), ObjectRef(a1651cc5330ea3b1ffffffff010000c001000000)]


In [ ]:
def fit_likelihood(action,states, p, MF, npars=2,x0=None, bounds=None):
#class ZO_DynamicLogit(object):
#    def __init__(self, data, Y, X, p, MF, npars):     

        #self.endog = data.loc[:, Y].values #Choice/action
        #self.exog = data.loc[:, X].values  #State
        self.endog = action
        self.exog = states
        self.N = self.endog.shape[0]
        self.S = int(91)
        
        # Check that p is a correct vector of probabilities (i.e. sums to 1)
        p = np.array(p)        
        if abs(p.sum()-1)<=0.005:
            self.p = p
        else:
            raise ValueError(("The probability of state transitions should add" 
                              " up to 1!"))
        
        
        # Check that the stated number of parameters correspond to the
        # specifications of the maintenance cost function.       
        try:
            MF(1, [0]*(npars-1))
        except ValueError:
            raise ValueError(("The number of parameters specified does not "
                              "match the specification of the maintenance cost"
                              " function!"))
        else:
            self.MF = MF
            self.npars = npars
        
        S = self.S
        
        # A (SxN) matrix indicating the state of each observation
 #       self.state_mat = np.array([[self.exog[i]==s for i in range(self.N)] 
 #                                                   for s in range(self.S)])
        
        # A (SxS) matrix indicating the probability of a bus transitioning
        # from a state s to a state s' (used to compute maintenance utility)
        
        self.trans_mat = np.zeros((S, S))
        for i in range(S):
            for j, _p in enumerate(self.p):
                if i + j < S-1:
                    self.trans_mat[i][i+j] = _p
                elif i + j == S-1:
                    self.trans_mat[i][S-1] = p[j:].sum()
                else:
                    pass

        # A second (SxS) matrix which regenerates the bus' state to 0 with
        # certainty (used to compute the replacement utility)
        self.regen_mat = np.vstack((np.ones((1, S)),np.zeros((S-1, S)))).T
       
        # A (2xN) matrix indicating with a dummy the decision taken by the agent
        # for each time/bus observation (replace or maintain)
#        self.dec_mat = np.vstack(((1-self.endog), self.endog)).T
    
    def myopic_costs(self, params): # - reward function
        S = self.S
        """
        This function computes the myopic expected cost associated with each 
        decision for each state.
        
        Takes:
            * A vector params, to be supplied to the maintenance cost function 
              MF. The first element of the vector is the replacement cost rc.

        Returns:
            * A (Sx2) array containing the maintenance and replacement costs 
              for the S possible states of the bus
        """
        rc = params[0]          #F : action 1
        thetas = params[1:]     #c : action 0
        maint_cost = [-self.MF(s, thetas) for s in range(0, S)]
        repl_cost = [-rc for s in range(0, S)]  #action 1
        return np.vstack((maint_cost, repl_cost)).T
    
    def fl_costs(self, params, beta=0.9999, threshold=1e-4, suppr_output=False): #compute V^n
        """
        Compute the non-myopic expected value of the agent for each possible 
        decision and each possible state of the bus, conditional on a vector of 
        parameters and on the maintenance cost function specified at the 
        initialization of the DynamicUtility model.

        Iterates until the difference in the previously obtained expected value 
        and the new expected value is smaller than a constant.
        
        Takes:
            * A vector params for the cost function
            * A discount factor beta (optional)
            * A convergence threshold (optional)
            * A boolean argument to suppress the output (optional)

        Returns:
            * An (Sx2) array of forward-looking costs associated with each
              state and each decision.
        """
        achieved = True
        # Initialization of the contraction mapping
        k = 0
        EV = np.ones((self.S, 1))        

        self.EV_myopic = self.myopic_costs(params)
        EV_new = np.zeros((self.S, 1))
       
        # Contraction mapping Loop
        while abs(EV_new-EV).max() > threshold:
            EV = EV_new 
            #pchoice = self.choice_prob(EV) #\pi_theta(s,a)
            Q0 = self.EV_myopic[:,0] + beta * self.trans_mat.dot(EV).reshape(-1)
            Q1 = self.EV_myopic[:,1] + beta * EV[0]
            Q = np.vstack((Q0,Q1)).T
            
            min_cost = Q.max(1).reshape(-1,1)
            cost = Q - min_cost
            util = np.exp(cost)
            EV_new =  min_cost + np.log(util.sum(1).reshape(-1,1))
            
            #ecost = (pchoice*EV).sum(1) 
            #futil_maint = np.dot(ecost, self.trans_mat)
            #futil_repl = np.dot(ecost, self.regen_mat)
            #futil = np.vstack((futil_maint, futil_repl)).T
            
            #EV_new = self.EV_myopic + beta*futil
            k += 1
            if k == 1000:
                achieved = False
                break

        # Output:
        if not suppr_output:
            if achieved:
                print("Convergence achieved in {} iterations".format(k))
            else:
                print("CM could not converge! Mean difference = {:.6f}".format(
                                                            (EV_new-EV).mean())
                                                                              )
        return EV_new,Q

    def choice_prob(self, cost_array):  #\pi_theta(s,a)
        """
        Returns the probability of each choice for each observed state, 
        conditional on an array of state/decision costs (generated by the 
        myopic or forward-looking cost functions)
        """
        cost = cost_array - cost_array.max(1).reshape(-1,1)
        util = np.exp(cost)
        pchoice = util/(np.sum(util, 1).reshape(-1,1))
        return pchoice
        
    def loglike(self, params):
        """
        The log-likelihood of the Dynamic model is estimated in several steps.
        1°) The current parameters are supplied to the contraction mapping 
            function
        2°) The function returns a matrix of decision probabilities for each 
            state.
        3°) This matrix is used to compute the loglikelihood of the 
            observations
        4°) The log-likelihood are then summed accross individuals, and 
            returned
        """
        utilV,utilQ = self.fl_costs(params, suppr_output=True) 
        pchoice = self.choice_prob(utilQ)    

        logprob = 0
        for sample_data in range(self.N):
            action = int(self.endog[sample_data])
            state = int(self.exog[sample_data])
            logprob += np.log(pchoice[state,action]) 
        return -logprob   
##################################Rust##################################    
#    def fit_likelihood(self, x0=None, bounds=None):
        """
        Fit the parameters to the data.
        """
        if bounds == None:
            bounds = [(1e-6, None) for i in range(self.npars)]
            
        if x0 == None:
            x0 = [0.1 for i in range(self.npars)]
            
        self.fitted = opt.fmin_l_bfgs_b(self.loglike, x0=x0, approx_grad=True, 
                                        bounds=bounds)
    '''
    def get_parameters(self):
        """
        Return the parameters obtained after fitting the likelihood function
        to the data.
        """
        return self.fitted[0]
        
    def print_parameters(self):
        loglike =  -self.fitted[1]
        fit_params = self.get_parameters()
        RC, thetas = fit_params[0], fit_params[1:]
        logstring = "Log-likelihood = {0:.4f}".format(loglike)
        thetas_string = ["theta1_{0} = {1:.6f}".format(i+1, t) \
                                                for i, t in enumerate(thetas)]
        thetas_string = ", ".join(thetas_string)
        rc_string = "Parameters: RC = {0:.6f}".format(RC)
        print(logstring, rc_string + ", " + thetas_string)    
    '''
        

[(0,), (1,), (2,)]
